In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from IPython.display import clear_output
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
columns = ['x_col']
df_images = pd.DataFrame(columns=columns)
df_masks = pd.DataFrame(columns=columns)

trainval_path = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\trainval.txt'
test_path = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\test.txt'

def get_image_mask_df(path):
    f = open(trainval_path,'r')
    image_names = []
    mask_names = []
    for i in f:
        image_name = i.split(' ')[0]
        image_names.append(['images\\'+ image_name + '.jpg'])
        mask_names.append(['annotations\\trimaps\\' + image_name+'.png'])
    
    
    df_images = pd.DataFrame(image_names,columns = columns)
    df_masks = pd.DataFrame(mask_names,columns = columns)
    return df_images, df_masks
df_train_images, df_train_masks = get_image_mask_df(trainval_path)
df_test_images, df_test_masks = get_image_mask_df(test_path)

In [ ]:
print(df_train_images.head())
print(df_train_masks.head())

In [ ]:
def preprocess_image(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    
    return input_image
def preprocess_mask(input_mask):
    input_mask -= 1
    return input_mask

def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
batch_size = 64

In [ ]:
diabetes_directory_images = 'C:\\Users\\tayyabm\\Downloads\\\Diabetes_Segmentation\\Original_Images\\Training_Set'
diabetes_directory_masks = 'C:\\Users\\tayyabm\\Downloads\\Diabetes_Segmentation\\All_Segmentation_Groundtruths\\Training_Set\\Microaneurysms'

In [ ]:
cats_directory = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation'

image_data_gen = ImageDataGenerator(preprocessing_function=preprocess_image)
mask_data_gen = ImageDataGenerator(preprocessing_function=preprocess_mask)
# train_image_generator = image_data_gen.flow_from_directory(diabetes_directory_images,target_size=(128,128),class_mode=None,shuffle=False,seed=0)
# train_mask_generator = image_data_gen.flow_from_directory(diabetes_directory_masks,target_size=(128,128),class_mode=None,shuffle=False,seed=0)
train_image_generator = image_data_gen.flow_from_dataframe(dataframe=df_train_images,x_col='x_col',directory=cats_directory,class_mode=None,target_size=(128,128),batch_size=batch_size, shuffle=False)
train_mask_generator = mask_data_gen.flow_from_dataframe(dataframe=df_train_masks,x_col='x_col',directory=cats_directory,class_mode=None,target_size=(128,128),batch_size=batch_size, shuffle=False)

In [ ]:
sample_image = tf.keras.preprocessing.image.array_to_img(train_image_generator.next()[0])
sample_mask = tf.keras.preprocessing.image.array_to_img(train_mask_generator.next()[0])

In [ ]:
train_mask_generator.next()[0].shape

In [ ]:
display([sample_image,sample_mask])

In [ ]:
train_generator = zip(train_image_generator, train_mask_generator)

In [ ]:
TRAIN_LENGTH = len(train_image_generator)*batch_size
BATCH_SIZE = batch_size
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
def encoding_base_block(input_tensor,filters,kernal_size,activation='relu',batch_normalization=True):
    x = input_tensor
    x = layers.Conv2D(filters=filters,kernel_size=kernal_size,padding='same',kernel_initializer='he_normal')(x)
    
    if batch_normalization:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(filters=filters,kernel_size=kernal_size,padding='same',kernel_initializer='he_normal')(x)
    
    if batch_normalization:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    return x

In [ ]:
def generate_unet(n_filter=16,dropout=0.1,batch_normalization =True):
    
    
    
    # encoding/reduction section
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs
    
    conv1 = encoding_base_block(input_tensor = x,filters=n_filter*1,kernal_size=(3,3),batch_normalization=batch_normalization)
    pool1 = layers.MaxPool2D((2, 2))(conv1)
    pool1 = layers.Dropout(dropout)(pool1)
    
    conv2 = encoding_base_block(input_tensor = pool1,filters=n_filter*2,kernal_size=(3,3),batch_normalization=batch_normalization)
    pool2 = layers.MaxPool2D((2,2))(conv2)
    pool2 = layers.Dropout(dropout)(pool2)
    
    conv3 = encoding_base_block(input_tensor = pool2,filters=n_filter*4,kernal_size=(3,3),batch_normalization=batch_normalization)
    pool3 = layers.MaxPool2D((2,2))(conv3)
    pool3 = layers.Dropout(dropout)(pool3)
    
    conv4 = encoding_base_block(input_tensor = pool3,filters=n_filter*8,kernal_size=(3,3),batch_normalization=batch_normalization)
    pool4 = layers.MaxPool2D((2,2))(conv4)
    pool4 = layers.Dropout(dropout)(pool4)
    
    conv5 = encoding_base_block(input_tensor=pool4,filters=n_filter*16,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    up6 = layers.Conv2DTranspose(n_filter*8,(3,3),strides=(2,2),padding='same')(conv5)
    up6 = layers.concatenate([up6,conv4])
    up6 = layers.Dropout(dropout)(up6)
    conv6 = encoding_base_block(input_tensor=up6,filters=n_filter*8,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    up7 = layers.Conv2DTranspose(n_filter*4,(3,3),strides=(2,2),padding='same')(conv6)
    up7 = layers.concatenate([up7,conv3])
    up7 = layers.Dropout(dropout)(up7)
    conv7 = encoding_base_block(input_tensor=up7,filters=n_filter*4,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    up8 = layers.Conv2DTranspose(n_filter*2,(3,3),strides=(2,2),padding='same')(conv7)
    up8 = layers.concatenate([up8,conv2])
    up8 = layers.Dropout(dropout)(up8)
    conv8 = encoding_base_block(input_tensor=up8,filters=n_filter*2,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    up9 = layers.Conv2DTranspose(n_filter*2,(3,3),strides=(2,2),padding='same')(conv8)
    up9 = layers.concatenate([up9,conv1])
    up9 = layers.Dropout(dropout)(up9)
    conv9 = encoding_base_block(input_tensor=up9,filters=n_filter,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    outputs = layers.Conv2D(3,(1,1),activation='sigmoid')(conv9)
    
    
    model = Model(inputs = inputs,outputs = [outputs])
    
    return model

In [ ]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

In [ ]:
@tf.function
def load_image_train(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [ ]:
model = None
model = generate_unet()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def create_mask(pred_mask):    
    pred_mask = tf.argmax(pred_mask, axis=-1)    
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        print('1 cleared_until_here')
        sample_image_input = tf.keras.preprocessing.image.img_to_array(sample_image)        
        print(sample_image_input.shape)
        sample_input_image=preprocess_image(sample_image_input)
        sample_input_image = np.expand_dims(sample_input_image,axis=0)
        display([sample_image, sample_mask,create_mask(model.predict(sample_input_image))])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
        print('4 clear here')
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
train_generator = (pair for pair in train_generator)
train_generator

In [ ]:
sample_image_input = tf.keras.preprocessing.image.img_to_array(sample_image)        

sample_input_image=preprocess_image(sample_image_input)
sample_input_image = np.expand_dims(sample_input_image,axis=0)

In [ ]:
display([sample_image, sample_mask])

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = 0.1*len(train_image_generator)

model_history = model.fit_generator(train_generator,epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,                          
                          callbacks=[DisplayCallback()]
                         )

In [ ]:
from PIL import Image

In [ ]:
im1 = Image.open('C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\trimaps\\Abyssinian_1.png')
im2 = Image.open('C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\images\\Abyssinian_1.jpg')

In [ ]:
print(im2.format)
plt.subplot(1, 2, 1)
plt.title('mask')
plt.imshow(im1)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('oriignal')
plt.imshow(im2)
plt.axis('off')


In [ ]:
plt.subplot(1, 2, 1)
plt.title('mask')
plt.imshow(im3)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('oriignal')
plt.imshow(im4)
plt.axis('off')

### UNET with PreTrained Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from IPython.display import clear_output
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def preprocess_image(input_image):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    
    return input_image
def preprocess_mask(input_mask):
    input_mask -= 1
    return input_mask

def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
columns = ['x_col']
df_images = pd.DataFrame(columns=columns)
df_masks = pd.DataFrame(columns=columns)

trainval_path = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\trainval.txt'
test_path = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\test.txt'

def get_image_mask_df(path):
    f = open(trainval_path,'r')
    image_names = []
    mask_names = []
    for i in f:
        image_name = i.split(' ')[0]
        image_names.append(['images\\'+ image_name + '.jpg'])
        mask_names.append(['annotations\\trimaps\\' + image_name+'.png'])
    
    
    df_images = pd.DataFrame(image_names,columns = columns)
    df_masks = pd.DataFrame(mask_names,columns = columns)
    return df_images, df_masks
df_train_images, df_train_masks = get_image_mask_df(trainval_path)
df_test_images, df_test_masks = get_image_mask_df(test_path)

In [ ]:
batch_size = 64

In [ ]:
cats_directory = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation'

image_data_gen = ImageDataGenerator(preprocessing_function=preprocess_image)
mask_data_gen = ImageDataGenerator(preprocessing_function=preprocess_mask)
# train_image_generator = image_data_gen.flow_from_directory(diabetes_directory_images,target_size=(128,128),class_mode=None,shuffle=False,seed=0)
# train_mask_generator = image_data_gen.flow_from_directory(diabetes_directory_masks,target_size=(128,128),class_mode=None,shuffle=False,seed=0)
train_image_generator = image_data_gen.flow_from_dataframe(dataframe=df_train_images,x_col='x_col',directory=cats_directory,class_mode=None,target_size=(128,128),batch_size=batch_size, shuffle=False,seed=1)
train_mask_generator = mask_data_gen.flow_from_dataframe(dataframe=df_train_masks,x_col='x_col',directory=cats_directory,class_mode=None,target_size=(128,128),batch_size=batch_size, shuffle=False,seed=1)

In [ ]:
sample_image = tf.keras.preprocessing.image.array_to_img(train_image_generator.next()[0])
sample_mask = tf.keras.preprocessing.image.array_to_img(train_mask_generator.next()[0])
display([sample_image,sample_mask])

In [ ]:
train_generator = zip(train_image_generator, train_mask_generator)
train_generator = (pair for pair in train_generator)

In [ ]:
TRAIN_LENGTH = len(train_image_generator)*batch_size
BATCH_SIZE = batch_size
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
def create_mask(pred_mask):    
    pred_mask = tf.argmax(pred_mask, axis=-1)    
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        print('1 cleared_until_here')
        sample_image_input = tf.keras.preprocessing.image.img_to_array(sample_image)        
        print(sample_image_input.shape)
        sample_input_image=preprocess_image(sample_image_input)
        sample_input_image = np.expand_dims(sample_input_image,axis=0)
        display([sample_image, sample_mask,create_mask(model_pretrained.predict(sample_input_image))])

In [ ]:
class DisplayCallbackPre(tf.keras.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs=None):        
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)
base_model.summary()

In [ ]:
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
down_stack.summary()

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [ ]:
up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):

  # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same', activation='softmax')  #64x64 -> 128x128

    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs

  # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model_pretrained = None
model_pretrained = unet_model(3)
model_pretrained.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
model_pretrained.summary()

In [ ]:
tf.keras.utils.plot_model(model_pretrained, show_shapes=True)

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = 0.1*len(train_image_generator)

model_history = model_pretrained.fit_generator(train_generator,epochs=EPOCHS,
                                               steps_per_epoch=STEPS_PER_EPOCH,                       
                                               callbacks=[DisplayCallbackPre()])

In [4]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras import Model

In [55]:
model_test=None
ip = Input(shape=(1,10,3))
conv = Conv2D(filters=1,kernel_size=(2,2),padding='same')(ip)
quant = QuantizeLayer()(conv)

model_test = Model(inputs = ip,outputs = quant)
model_test.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 1, 10, 3)]        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 1, 10, 1)          13        
_________________________________________________________________
quantize_layer_11 (QuantizeL (None, 1, 10, 1)          0         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [58]:
oo = 100*tf.ones(
    (1,1,10,3),
    dtype=tf.dtypes.float32,
    name=None
)
model_test.predict(oo)

array([[[[  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [  2.3125],
         [-22.1875]]]], dtype=float32)

### UNET with Quantization

In [1]:
import tensorflow as tf

/home/mujahid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
from tensorflow.python.framework import ops

In [ ]:
def log1pexp(x):
    return tf.math.log(1 + tf.exp(x))

In [ ]:
x = tf.constant(100.)
print(x)
y = log1pexp(x)
y.numpy()

In [ ]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.math.log(1 + e), grad

In [ ]:
x = tf.constant(100.)
y = log1pexp(x)

In [ ]:
y

In [35]:
tf.math.log(1 + tf.exp(100.))
from tensorflow.keras import backend as K

In [52]:

def clamp(input_tensor, num_bits=4):
    input_tensor = tf.clip_by_value(input_tensor, clip_value_min=0, clip_value_max=2 ** (num_bits) - 1)
    input_tensor = ((tf.math.round(input_tensor * 2 ** (num_bits)))) / 2 ** (num_bits)
    return input_tensor


def to_fixed_point(input_tensor, ibits, fbits):
    input_tensor_i = K.sign(input_tensor) * clamp(tf.floor(tf.abs(input_tensor)), ibits)
    input_tensor_f = K.sign(input_tensor) * clamp(K.abs(input_tensor) - tf.floor(K.abs(input_tensor)), fbits)
    return input_tensor_i + input_tensor_f

In [53]:
@tf.custom_gradient
def quantize(x):
    y = to_fixed_point(x, ibits=6, fbits=4)

    def _fixed_point_grad(dy):
        grad = dy
        return grad

    return y, _fixed_point_grad


class QuantizeLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantizeLayer, self).__init__()

    def call(self, x):
        return quantize(x)

In [54]:
q_layer = QuantizeLayer()

In [ ]:
q_layer(tf.Variable(tf.constant([124,3,],dtype=tf.float32)))

In [ ]:
input1 = Input(shape=(1,4))